# 🍅 Tomato Leaf Disease Detection - YOLOv11 Training (20 Epochs + Augmentation)

This notebook trains a YOLOv11 model on PlantVillage tomato disease dataset with robust data augmentation.

**Target Classes:**
1. Tomato_Bacterial_spot
2. Tomato_Early_blight
3. Tomato_Late_blight
4. Tomato_Septoria_leaf_spot
5. Tomato_Tomato_mosaic_virus
6. Tomato_healthy

**Training Configuration:**
- Epochs: 20
- Image Size: 640
- Batch Size: 16
- Model: YOLOv11n (nano)

**Data Augmentation (Real-World Robustness):**
- Rotation: ±15 degrees
- Horizontal flip: 50% probability
- Zoom/Scale: ±10%
- Brightness: ±20% variation
- Random crops: Various positions
- HSV color jitter for lighting variations

## 📋 Step 1: Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q ultralytics kaggle opencv-python-headless

# Import libraries
import os
import shutil
from pathlib import Path
from google.colab import files
import yaml
from ultralytics import YOLO

print("✅ Environment setup complete!")

## 🔑 Step 2: Setup Kaggle API Credentials

In [ ]:
# Upload your kaggle.json file
print("📤 Please upload your kaggle.json file")
print("   Get it from: https://www.kaggle.com/account")
uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API configured!")

## 📥 Step 3: Download PlantVillage Dataset

In [ ]:
# Download PlantVillage dataset from Kaggle
!kaggle datasets download -d arjuntejaswi/plant-village --unzip -p /content/plantvillage

print("✅ Dataset downloaded!")

## 🍅 Step 4: Filter Tomato Classes

In [ ]:
# Define target tomato classes
TOMATO_CLASSES = [
    'Tomato_Bacterial_spot',
    'Tomato_Early_blight',
    'Tomato_Late_blight',
    'Tomato_Septoria_leaf_spot',
    'Tomato_Tomato_mosaic_virus',  # Note: Double 'Tomato_' is correct!
    'Tomato_healthy'
]

# Alternative class names to try if primary name not found
ALTERNATIVE_NAMES = {
    'Tomato_Tomato_mosaic_virus': ['Tomato_mosaic_virus', 'Tomato__Tomato_mosaic_virus', 'Tomato___Tomato_mosaic_virus']
}

# Find dataset root
dataset_root = Path('/content/plantvillage')
possible_roots = [
    dataset_root / 'PlantVillage',
    dataset_root / 'New Plant Diseases Dataset(Augmented)' / 'New Plant Diseases Dataset(Augmented)',
    dataset_root
]

source_dir = None
for root in possible_roots:
    if root.exists():
        tomato_folders = [f for f in root.iterdir() if f.is_dir() and 'Tomato' in f.name]
        if tomato_folders:
            source_dir = root
            break

if source_dir is None:
    # Search recursively
    for item in dataset_root.rglob('*'):
        if item.is_dir() and 'Tomato' in item.name:
            source_dir = item.parent
            break

print(f"📁 Dataset found at: {source_dir}")

# Filter and copy tomato classes
filtered_dir = Path('/content/tomato_filtered')
filtered_dir.mkdir(exist_ok=True)

stats = {}
for class_name in TOMATO_CLASSES:
    source_class = source_dir / class_name
    
    # Try main name first
    if source_class.exists():
        dest_class = filtered_dir / class_name
        shutil.copytree(source_class, dest_class, dirs_exist_ok=True)
        
        image_count = len(list(dest_class.glob('*.jpg'))) + len(list(dest_class.glob('*.JPG')))
        stats[class_name] = image_count
        print(f"✅ {class_name}: {image_count} images")
    # Try alternative names
    elif class_name in ALTERNATIVE_NAMES:
        found = False
        for alt_name in ALTERNATIVE_NAMES[class_name]:
            alt_source = source_dir / alt_name
            if alt_source.exists():
                dest_class = filtered_dir / class_name
                shutil.copytree(alt_source, dest_class, dirs_exist_ok=True)
                
                image_count = len(list(dest_class.glob('*.jpg'))) + len(list(dest_class.glob('*.JPG')))
                stats[class_name] = image_count
                print(f"✅ {class_name}: {image_count} images (found as {alt_name})")
                found = True
                break
        if not found:
            print(f"⚠️  {class_name}: Not found in dataset")
    else:
        print(f"⚠️  {class_name}: Not found in dataset")

# Update TOMATO_CLASSES to only include found classes
TOMATO_CLASSES = list(stats.keys())
print(f"\n📊 Total: {sum(stats.values())} images across {len(stats)} classes")
print(f"\n📋 Final classes for training: {TOMATO_CLASSES}")

## 🎨 Step 5: Apply Comprehensive Manual Augmentation

This step creates augmented versions of training images to expand the dataset and improve model robustness.

In [ ]:
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import cv2

def apply_augmentation(img, aug_type):
    """
    Apply specific augmentation to an image.
    
    Augmentation types:
    - rotate_15: Rotate ±15 degrees
    - flip_h: Horizontal flip
    - zoom_in: Zoom in 10%
    - zoom_out: Zoom out 10%
    - bright: Increase brightness 20%
    - dark: Decrease brightness 20%
    - crop_tl: Crop from top-left
    - crop_tr: Crop from top-right
    - crop_bl: Crop from bottom-left
    - crop_br: Crop from bottom-right
    - crop_center: Crop from center
    """
    img_array = np.array(img)
    h, w = img_array.shape[:2]
    
    if aug_type == 'rotate_15':
        angle = random.uniform(-15, 15)
        center = (w // 2, h // 2)
        matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(img_array, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)
        return Image.fromarray(rotated)
    
    elif aug_type == 'flip_h':
        return img.transpose(Image.FLIP_LEFT_RIGHT)
    
    elif aug_type == 'zoom_in':
        # Zoom in 10% (crop center and resize)
        crop_size = int(min(w, h) * 0.9)
        left = (w - crop_size) // 2
        top = (h - crop_size) // 2
        cropped = img.crop((left, top, left + crop_size, top + crop_size))
        return cropped.resize((w, h), Image.LANCZOS)
    
    elif aug_type == 'zoom_out':
        # Zoom out 10% (add padding)
        new_size = int(min(w, h) * 1.1)
        resized = img.resize((int(w * 0.9), int(h * 0.9)), Image.LANCZOS)
        new_img = Image.new('RGB', (w, h), (0, 0, 0))
        paste_x = (w - resized.width) // 2
        paste_y = (h - resized.height) // 2
        new_img.paste(resized, (paste_x, paste_y))
        return new_img
    
    elif aug_type == 'bright':
        enhancer = ImageEnhance.Brightness(img)
        return enhancer.enhance(1.2)  # 20% brighter
    
    elif aug_type == 'dark':
        enhancer = ImageEnhance.Brightness(img)
        return enhancer.enhance(0.8)  # 20% darker
    
    elif aug_type.startswith('crop_'):
        # Random crop from different positions (90% of image)
        crop_size = int(min(w, h) * 0.9)
        
        if aug_type == 'crop_tl':  # Top-left
            left, top = 0, 0
        elif aug_type == 'crop_tr':  # Top-right
            left, top = w - crop_size, 0
        elif aug_type == 'crop_bl':  # Bottom-left
            left, top = 0, h - crop_size
        elif aug_type == 'crop_br':  # Bottom-right
            left, top = w - crop_size, h - crop_size
        else:  # crop_center
            left = (w - crop_size) // 2
            top = (h - crop_size) // 2
        
        cropped = img.crop((left, top, left + crop_size, top + crop_size))
        return cropped.resize((w, h), Image.LANCZOS)
    
    return img

# Create augmented dataset directory
augmented_dir = Path('/content/tomato_augmented')
augmented_dir.mkdir(exist_ok=True)

# Define augmentation strategies
AUGMENTATIONS = [
    'rotate_15',
    'flip_h',
    'zoom_in',
    'bright',
    'dark',
    'crop_center'
]

print("🎨 Applying comprehensive augmentation...")
print(f"   Augmentation types: {len(AUGMENTATIONS)}")
print(f"   Expected dataset expansion: {len(AUGMENTATIONS) + 1}x\n")

aug_stats = {}
for class_name in TOMATO_CLASSES:
    source_class = filtered_dir / class_name
    dest_class = augmented_dir / class_name
    dest_class.mkdir(exist_ok=True)
    
    images = list(source_class.glob('*.jpg')) + list(source_class.glob('*.JPG'))
    original_count = len(images)
    augmented_count = 0
    
    for img_path in images:
        try:
            img = Image.open(img_path).convert('RGB')
            
            # Save original
            orig_dest = dest_class / f"{img_path.stem}_orig.jpg"
            img.save(orig_dest, quality=95)
            augmented_count += 1
            
            # Apply each augmentation
            for aug_type in AUGMENTATIONS:
                aug_img = apply_augmentation(img, aug_type)
                aug_dest = dest_class / f"{img_path.stem}_{aug_type}.jpg"
                aug_img.save(aug_dest, quality=95)
                augmented_count += 1
        
        except Exception as e:
            print(f"⚠️  Error processing {img_path.name}: {e}")
    
    aug_stats[class_name] = {
        'original': original_count,
        'augmented': augmented_count
    }
    print(f"✅ {class_name}: {original_count} → {augmented_count} images ({augmented_count/original_count:.1f}x)")

total_original = sum(s['original'] for s in aug_stats.values())
total_augmented = sum(s['augmented'] for s in aug_stats.values())
print(f"\n📊 Total: {total_original} → {total_augmented} images ({total_augmented/total_original:.1f}x expansion)")
print("✅ Comprehensive augmentation complete!")

## 📦 Step 6: Convert Augmented Dataset to YOLO Format

In [ ]:
import random
from PIL import Image

# Create YOLO directory structure
yolo_dir = Path('/content/tomato_yolo')
for split in ['train', 'val', 'test']:
    (yolo_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (yolo_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# Process each class from augmented dataset
class_mapping = {name: idx for idx, name in enumerate(TOMATO_CLASSES)}
split_counts = {'train': 0, 'val': 0, 'test': 0}

for class_name, class_id in class_mapping.items():
    class_dir = augmented_dir / class_name
    images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.JPG'))
    random.shuffle(images)
    
    n_train = int(len(images) * TRAIN_RATIO)
    n_val = int(len(images) * VAL_RATIO)
    
    splits = {
        'train': images[:n_train],
        'val': images[n_train:n_train+n_val],
        'test': images[n_train+n_val:]
    }
    
    for split_name, split_images in splits.items():
        for img_path in split_images:
            # Copy image
            img_name = f"{class_name}_{img_path.stem}{img_path.suffix}"
            dest_img = yolo_dir / split_name / 'images' / img_name
            shutil.copy2(img_path, dest_img)
            
            # Create label (full image bounding box)
            label_name = f"{class_name}_{img_path.stem}.txt"
            label_path = yolo_dir / split_name / 'labels' / label_name
            with open(label_path, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            split_counts[split_name] += 1

print("📊 Augmented Dataset Split:")
print(f"   Train: {split_counts['train']} images")
print(f"   Val: {split_counts['val']} images")
print(f"   Test: {split_counts['test']} images")
print(f"   Total: {sum(split_counts.values())} images")
print("✅ YOLO format conversion complete!")

## 📝 Step 7: Create Dataset YAML

In [ ]:
# Create dataset.yaml
dataset_yaml = {
    'path': str(yolo_dir),
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': len(TOMATO_CLASSES),
    'names': TOMATO_CLASSES
}

yaml_path = yolo_dir / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(f"✅ Dataset YAML created at: {yaml_path}")
print("\n📄 Content:")
print(yaml.dump(dataset_yaml, default_flow_style=False))

## 🚀 Step 8: Train YOLOv11 Model (20 Epochs + Runtime Augmentation)

In [ ]:
# Initialize YOLOv11n model
model = YOLO('yolo11n.pt')

# Train the model with comprehensive augmentation for real-world robustness
# Augmentation strategy:
# - degrees=15: Rotate images ±15 degrees (handles different camera angles)
# - scale=0.1: Zoom in/out by ±10% (handles varying distances)
# - translate=0.15: Random crops at different positions (handles leaf positioning)
# - fliplr=0.5: Horizontal flip 50% of time (handles leaf orientation)
# - flipud=0.0: No vertical flip (leaves don't grow upside down)
# - hsv_v=0.2: Brightness variation ±20% (handles lighting conditions)
# - hsv_s=0.5: Saturation variation (handles color differences)
# - hsv_h=0.01: Slight hue shift (maintains leaf color integrity)
# - mosaic=0.0: Disabled (not suitable for full-image classification)
# - mixup=0.0: Disabled (not suitable for full-image classification)

results = model.train(
    data=str(yaml_path),
    epochs=20,              # Reduced to 20 epochs as requested
    imgsz=640,
    batch=16,
    device=0,
    name='tomato_disease_yolo11n_20ep',
    augment=True,           # Enable augmentation
    degrees=15.0,           # ±15° rotation for angle variations
    scale=0.1,              # ±10% zoom for distance variations
    translate=0.15,         # Random crops at different positions
    fliplr=0.5,             # 50% horizontal flip probability
    flipud=0.0,             # No vertical flip (unrealistic)
    hsv_h=0.01,             # Minimal hue shift (keep leaf color natural)
    hsv_s=0.5,              # Moderate saturation variation
    hsv_v=0.2,              # ±20% brightness (darker/brighter lighting)
    mosaic=0.0,             # Disabled - not suitable for classification
    mixup=0.0,              # Disabled - not suitable for classification
    copy_paste=0.0,         # Disabled - not suitable for classification
    perspective=0.0,        # Disabled to maintain leaf shape integrity
    shear=0.0               # Disabled to maintain leaf shape integrity
)

print("✅ Training complete with robust augmentation!")
print("\n📊 Augmentation Summary:")
print("   ✓ Rotation: ±15° (angle robustness)")
print("   ✓ Horizontal flip: 50% (orientation robustness)")
print("   ✓ Zoom: ±10% (distance robustness)")
print("   ✓ Brightness: ±20% (lighting robustness)")
print("   ✓ Random crops: Various positions (leaf position robustness)")
print("   ✓ Color jitter: HSV variations (real-world color robustness)")
print("\n⏭️  Next: Run Step 9 (Validate), Step 10 (Export), then Step 11 (Auto-Download)")

## 📊 Step 9: Validate Model

In [ ]:
# Validate on test set
metrics = model.val(data=str(yaml_path), split='test')

print("\n📊 Validation Metrics:")
print(f"   mAP50: {metrics.box.map50:.4f}")
print(f"   mAP50-95: {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall: {metrics.box.mr:.4f}")

## 💾 Step 10: Export Model to Multiple Formats

In [ ]:
print("📤 Exporting model to multiple formats...\n")

# Export to TFLite (INT8 quantized for mobile)
print("1️⃣ Exporting to TFLite INT8 (quantized, smallest size)...")
model.export(format='tflite', int8=True, data=str(yaml_path))
print("   ✅ TFLite INT8 exported!\n")

# Export to TFLite Float16 (good balance)
print("2️⃣ Exporting to TFLite Float16 (balanced)...")
try:
    model.export(format='tflite', half=True, data=str(yaml_path))
    print("   ✅ TFLite Float16 exported!\n")
except:
    print("   ⚠️  Float16 export not available\n")

# Export to TFLite Float32 (highest accuracy)
print("3️⃣ Exporting to TFLite Float32 (highest accuracy)...")
try:
    model.export(format='tflite', data=str(yaml_path))
    print("   ✅ TFLite Float32 exported!\n")
except:
    print("   ⚠️  Float32 export not available\n")

print("✅ All model exports complete!")
print("\n📊 Model Formats:")
print("   • best.pt - PyTorch model (for Python inference)")
print("   • best_int8.tflite - Quantized TFLite (smallest, for Android)")
print("   • best_float16.tflite - Half precision (balanced)")
print("   • best_float32.tflite - Full precision (highest accuracy)")
print("\n⏭️  Next: Run Step 11 to automatically download all files!")

## 📥 Step 11: Automatically Download All Trained Models & Results

This cell automatically downloads all important files to your computer after training.

In [ ]:
import time
import zipfile

print("📦 Preparing files for automatic download...\n")

# Define all files to download
run_dir = Path('/content/runs/detect/tomato_disease_yolo11n_20ep')
weights_dir = run_dir / 'weights'
tflite_dir = weights_dir / 'best_saved_model'

files_to_download = [
    # PyTorch models
    (weights_dir / 'best.pt', 'best.pt', 'Best PyTorch model'),
    (weights_dir / 'last.pt', 'last.pt', 'Last epoch PyTorch model'),
    
    # TFLite models
    (tflite_dir / 'best_int8.tflite', 'best_int8.tflite', 'Quantized TFLite model (INT8)'),
    (tflite_dir / 'best_float32.tflite', 'best_float32.tflite', 'Float32 TFLite model'),
    (tflite_dir / 'best_float16.tflite', 'best_float16.tflite', 'Float16 TFLite model'),
    
    # Training results
    (run_dir / 'results.png', 'results.png', 'Training curves'),
    (run_dir / 'results.csv', 'results.csv', 'Training metrics CSV'),
    (run_dir / 'confusion_matrix.png', 'confusion_matrix.png', 'Confusion matrix'),
    (run_dir / 'confusion_matrix_normalized.png', 'confusion_matrix_normalized.png', 'Normalized confusion matrix'),
    
    # Additional metrics
    (run_dir / 'F1_curve.png', 'F1_curve.png', 'F1 score curve'),
    (run_dir / 'PR_curve.png', 'PR_curve.png', 'Precision-Recall curve'),
    (run_dir / 'P_curve.png', 'P_curve.png', 'Precision curve'),
    (run_dir / 'R_curve.png', 'R_curve.png', 'Recall curve'),
]

# Download each file individually
downloaded_files = []
for file_path, display_name, description in files_to_download:
    if file_path.exists():
        try:
            files.download(str(file_path))
            downloaded_files.append(display_name)
            print(f"✅ Downloaded: {display_name} ({description})")
            time.sleep(0.5)  # Small delay between downloads
        except Exception as e:
            print(f"⚠️  Failed to download {display_name}: {e}")
    else:
        print(f"⚠️  Not found: {display_name}")

# Create a comprehensive ZIP archive
print("\n📦 Creating comprehensive ZIP archive...")
zip_path = Path('/content/tomato_yolo11n_20ep_complete.zip')

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all training outputs
    for file_path, display_name, description in files_to_download:
        if file_path.exists():
            zipf.write(file_path, f'tomato_yolo11n_20ep/{display_name}')
    
    # Add dataset YAML
    yaml_file = yolo_dir / 'dataset.yaml'
    if yaml_file.exists():
        zipf.write(yaml_file, 'tomato_yolo11n_20ep/dataset.yaml')
    
    # Add sample predictions if available
    predict_dir = Path('/content/runs/detect/predict')
    if predict_dir.exists():
        for img in list(predict_dir.glob('*.jpg'))[:5]:
            zipf.write(img, f'tomato_yolo11n_20ep/sample_predictions/{img.name}')

# Download the complete ZIP
if zip_path.exists():
    print(f"\n📥 Downloading complete archive: {zip_path.name}")
    files.download(str(zip_path))
    print(f"✅ Complete! Archive size: {zip_path.stat().st_size / (1024*1024):.1f} MB")

print(f"\n🎉 All files downloaded successfully!")
print(f"   Individual files: {len(downloaded_files)}")
print(f"   Complete archive: tomato_yolo11n_20ep_complete.zip")
print(f"\n💡 Tip: The ZIP archive contains everything in one convenient package!")

## 🎯 Step 12: Test Inference

In [ ]:
# Test on a sample image
test_images = list((yolo_dir / 'test' / 'images').glob('*.jpg'))[:5]

for test_img in test_images:
    results = model.predict(source=str(test_img), save=True, conf=0.5)
    print(f"✅ Processed: {test_img.name}")

print("\n📁 Results saved to: /content/runs/detect/predict")

## 🚀 AUTOMATED: Validate, Export & Download (Run This After Training!)

**This cell automatically:**
1. Validates the model on test set
2. Exports to all TFLite formats
3. Downloads all files to your computer

Just run this one cell after training completes!

In [ ]:
import time
import zipfile
from IPython.display import Image as IPImage, display

print("🚀 Starting automated post-training workflow...\n")
print("=" * 60)

# ============================================================================
# STEP 1: VALIDATE MODEL
# ============================================================================
print("\n📊 STEP 1: Validating model on test set...\n")
metrics = model.val(data=str(yaml_path), split='test')

print("\n✅ Validation Metrics:")
print(f"   • mAP50: {metrics.box.map50:.4f}")
print(f"   • mAP50-95: {metrics.box.map:.4f}")
print(f"   • Precision: {metrics.box.mp:.4f}")
print(f"   • Recall: {metrics.box.mr:.4f}")

# ============================================================================
# STEP 2: EXPORT TO MULTIPLE FORMATS
# ============================================================================
print("\n" + "=" * 60)
print("\n📤 STEP 2: Exporting model to multiple formats...\n")

# Export to TFLite INT8
print("1️⃣ Exporting to TFLite INT8 (quantized, smallest size)...")
try:
    model.export(format='tflite', int8=True, data=str(yaml_path))
    print("   ✅ TFLite INT8 exported!")
except Exception as e:
    print(f"   ⚠️  INT8 export failed: {e}")

# Export to TFLite Float16
print("\n2️⃣ Exporting to TFLite Float16 (balanced)...")
try:
    model.export(format='tflite', half=True, data=str(yaml_path))
    print("   ✅ TFLite Float16 exported!")
except Exception as e:
    print(f"   ⚠️  Float16 export failed: {e}")

# Export to TFLite Float32
print("\n3️⃣ Exporting to TFLite Float32 (highest accuracy)...")
try:
    model.export(format='tflite', data=str(yaml_path))
    print("   ✅ TFLite Float32 exported!")
except Exception as e:
    print(f"   ⚠️  Float32 export failed: {e}")

# ============================================================================
# STEP 3: TEST INFERENCE
# ============================================================================
print("\n" + "=" * 60)
print("\n🎯 STEP 3: Running test inference on sample images...\n")

test_images = list((yolo_dir / 'test' / 'images').glob('*.jpg'))[:5]
for test_img in test_images:
    results = model.predict(source=str(test_img), save=True, conf=0.5)
    print(f"   ✅ Processed: {test_img.name}")

# ============================================================================
# STEP 4: DISPLAY RESULTS
# ============================================================================
print("\n" + "=" * 60)
print("\n📊 STEP 4: Displaying training results...\n")

# Display training curves
results_path = Path('/content/runs/detect/tomato_disease_yolo11n_20ep/results.png')
if results_path.exists():
    print("📈 Training Curves:")
    display(IPImage(filename=str(results_path)))

# Display confusion matrix
confusion_matrix_path = Path('/content/runs/detect/tomato_disease_yolo11n_20ep/confusion_matrix.png')
if confusion_matrix_path.exists():
    print("\n📊 Confusion Matrix:")
    display(IPImage(filename=str(confusion_matrix_path)))

# ============================================================================
# STEP 5: AUTOMATIC DOWNLOAD
# ============================================================================
print("\n" + "=" * 60)
print("\n📥 STEP 5: Automatically downloading all files to your computer...\n")

run_dir = Path('/content/runs/detect/tomato_disease_yolo11n_20ep')
weights_dir = run_dir / 'weights'
tflite_dir = weights_dir / 'best_saved_model'

files_to_download = [
    (weights_dir / 'best.pt', 'best.pt', 'Best PyTorch model'),
    (weights_dir / 'last.pt', 'last.pt', 'Last epoch model'),
    (tflite_dir / 'best_int8.tflite', 'best_int8.tflite', 'INT8 TFLite'),
    (tflite_dir / 'best_float32.tflite', 'best_float32.tflite', 'Float32 TFLite'),
    (tflite_dir / 'best_float16.tflite', 'best_float16.tflite', 'Float16 TFLite'),
    (run_dir / 'results.png', 'results.png', 'Training curves'),
    (run_dir / 'results.csv', 'results.csv', 'Metrics CSV'),
    (run_dir / 'confusion_matrix.png', 'confusion_matrix.png', 'Confusion matrix'),
]

# Download individual files
downloaded_count = 0
for file_path, display_name, description in files_to_download:
    if file_path.exists():
        try:
            files.download(str(file_path))
            print(f"   ✅ {display_name} ({description})")
            downloaded_count += 1
            time.sleep(0.3)
        except Exception as e:
            print(f"   ⚠️  Failed: {display_name}")

# Create and download ZIP archive
print("\n📦 Creating complete ZIP archive...")
zip_path = Path('/content/tomato_yolo11n_20ep_complete.zip')

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file_path, display_name, description in files_to_download:
        if file_path.exists():
            zipf.write(file_path, f'tomato_yolo11n_20ep/{display_name}')
    
    # Add dataset YAML
    yaml_file = yolo_dir / 'dataset.yaml'
    if yaml_file.exists():
        zipf.write(yaml_file, 'tomato_yolo11n_20ep/dataset.yaml')
    
    # Add sample predictions
    predict_dir = Path('/content/runs/detect/predict')
    if predict_dir.exists():
        for img in list(predict_dir.glob('*.jpg'))[:5]:
            zipf.write(img, f'tomato_yolo11n_20ep/predictions/{img.name}')

if zip_path.exists():
    files.download(str(zip_path))
    zip_size = zip_path.stat().st_size / (1024*1024)
    print(f"   ✅ Complete archive ({zip_size:.1f} MB)")

# ============================================================================
# COMPLETION SUMMARY
# ============================================================================
print("\n" + "=" * 60)
print("\n🎉 AUTOMATED WORKFLOW COMPLETE!\n")
print("📊 Summary:")
print(f"   ✅ Model validated (mAP50: {metrics.box.map50:.4f})")
print(f"   ✅ Exported to 3 TFLite formats")
print(f"   ✅ Downloaded {downloaded_count} individual files")
print(f"   ✅ Downloaded complete ZIP archive")
print("\n💡 All files are now saved to your computer!")
print("   Use 'best_int8.tflite' for Android deployment.")
print("\n" + "=" * 60)

## 📊 Step 13: Display Training Results (Optional - Already shown above)

In [ ]:
# This cell is optional - results are already displayed in the automated workflow above
from IPython.display import Image as IPImage, display

# Display training curves
results_path = Path('/content/runs/detect/tomato_disease_yolo11n_20ep/results.png')
if results_path.exists():
    display(IPImage(filename=str(results_path)))

# Display confusion matrix
confusion_matrix_path = Path('/content/runs/detect/tomato_disease_yolo11n_20ep/confusion_matrix.png')
if confusion_matrix_path.exists():
    display(IPImage(filename=str(confusion_matrix_path)))

## ✅ Training Complete!

Your YOLOv11 model has been trained for 20 epochs with comprehensive dual-layer data augmentation on the 6 tomato disease classes.

**Comprehensive Augmentation Strategy:**

*Pre-Training (Manual) Augmentation:*
- ✓ Rotation: ±15° variations
- ✓ Horizontal flip: Mirror images
- ✓ Zoom in: 10% closer view
- ✓ Brightness: +20% (brighter conditions)
- ✓ Darkness: -20% (darker conditions)
- ✓ Center crop: Different leaf positions
- ✓ Dataset expanded ~7x before training

*Runtime (Training-Time) Augmentation:*
- ✓ Additional rotation: ±15° random
- ✓ Random horizontal flip: 50%
- ✓ Scale variation: ±10%
- ✓ Translation: ±15% random crops
- ✓ HSV color jitter: Brightness, saturation, hue variations

**Benefits:**
- Massive dataset expansion (7x original size)
- Dual-layer augmentation (pre-training + runtime)
- Significantly reduced overfitting
- Excellent generalization to real-world conditions
- Robust to lighting, angles, distances, and leaf positions

**Files Generated:**
- `best.pt` - Best PyTorch model
- `best_int8.tflite` - Quantized TFLite model for mobile deployment
- `results.png` - Training metrics visualization
- `confusion_matrix.png` - Model performance analysis

**Next Steps:**
1. Download the models using the cells above
2. Integrate into your Android app
3. Test with real tomato leaf images